In [1]:
import requests
import json
import pandas as pd
import time
import numpy as np
from threading import Thread, Lock, Event
import concurrent.futures as ft
api_key = "your api key"
def findPlace(businessName,address, apiKey):
    # output - list of json
    json_response_list = []
    base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
    tempInput = (businessName+' ' + address).replace(" ","%20").lower()
   
    parameter_url = f'input={tempInput}&inputtype=textquery&fields=business_status,place_id,name&key={apiKey}'#,formatted_address,geometry,icon,name,place_id,plus_code,types
    
    url = base_url + parameter_url
    #print(url)
    try:
        response = requests.get(url)
        json_response = response.json()
    except:
        try:
            response = requests.get(url)
            json_response = response.json()
        except:
            json_response = {'candidates': '',
                             'status': 'Request Failed'}

    return(json_response)

def findPlaceDetails(placeID, apiKey):
    parameter_url = 'https://maps.googleapis.com/maps/api/place/details/json?place_id='+ placeID + '&fields=name,rating,formatted_phone_number,website&key=' + apiKey
    try:
        response = requests.get(parameter_url)
        json_response = response.json()
    except:
        json_response = {'candidates': '',
                         'status': 'Request Failed'}
    return(json_response)


In [2]:
df = pd.read_csv('VZ_leads_MM_MB_COMBINED_3P.csv')

In [3]:
df.head(50)

COMPANY  \
0                                 ALEX BROWN REALTY   
1   AMTTHE ASSOCIATION FOR MANUFACTURING TECHNOLOGY   
2                                   BULBTRONICS INC   
3                                 LONG BOW RESEARCH   
4                           PLC MEDICAL SYSTEMS INC   
5                    POTOMAC RES GROUP HOLDINGS LLC   
6                            WALEK  ASSOCIATES  INC   
7              NATIONAL MUSEUM OF WOMEN IN THE ARTS   
8                              BRAHMAN CAPITAL CORP   
9                                  DERLE FARMS  INC   
10                               ALPHA GARMENT  INC   
11                       LAWLER TERRACE CORPORATION   
12                    CHROME HEARTS OF NEW YORK INC   
13                            DIRORY INDUSTRIES INC   
14                      JLB PARTNERS MANAGEMENT LLC   
15                           AEC REPROGRAPHICS  INC   
16                  INC RICHARD LEEDS INTERNATIONAL   
17         MASSACHUSETTS CONTINUING LEGAL EDUCATION   
18                                  CABLE SCOPE INC   
19                           ESCO FASTENERS CO  INC   
20                    INSURANCE  FINANCIAL SVCS INC   
21                       KUMI CONSTRUCTION MGT CORP   
22                                    L TRONICS INC   
23                                    NETOLOGIC INC   
24                       NETWORK AMERICA LINES  INC   
25                          PROPERTY GROUP PARTNERS   
26                             ZEPHYR ALUMINUM  LLC   
27                   INTERNAL MEDICINE PHYSICIANS O   
28                                  LASERMATION INC   
29                                        TACO BELL   
30                             BRONSON NUTRITIONALS   
31                   CHINATOWN MANPOWER PROJECT INC   
32                             EAST  ASSOCIATES  LP   
33                             AAM MOTOR SALES  LLC   
34                                ROYAL IMPORTS INC   
35                     DELEON  STANG CPAS  ADVISORS   
36                    MILLEN  WHITE  ZELANO  BRANIG   
37                                  SITE SAFETY LLC   
38                                  MANHATTAN SIGNS   
39                           POWER SYSTEMS ELECTRIC   
40                                 DRAGON ROUGE LLC   
41                                            REMAX   
42               STRUCTURAL ASSESSMENT HOLDINGS INC   
43                                      WARTHEN LLC   
44                       WITTMAN PLUMBING ASSOC INC   
45                               BRUNE  RICHARD LLP   
46                                        VDRNC LLC   
47                           ARTIST POINT GIFT WRAP   
48                      QUINN FABLE ADVERTISING INC   
49                        WAYNE PRESBYTERIAN CHURCH   

                                STREET           CITY STATE  \
0            300 E LOMBARD ST STE 1200      BALTIMORE    MD   
1         7901 JONES BRANCH DR STE 900        MC LEAN    VA   
2                      45 BANFI PLZ N     FARMINGDALE    NY   
3                        200 PARK AVE        NEW YORK    NY   
4                    459 FORTUNE BLVD         MILFORD    MA   
5   600 NEW HAMPSHIRE AVE NW UNIT 1000     WASHINGTON    DC   
6                 470 PARK AVE S FL 4N       NEW YORK    NY   
7                1250 NEW YORK AVE NW      WASHINGTON    DC   
8                655 3RD AVE UNIT 1100       NEW YORK    NY   
9             15 GRUMMAN RD W UNIT 100       BETHPAGE    NY   
10                  1385 BROADWAY FL 5       NEW YORK    NY   
11                  10300 DRUMMOND RD    PHILADELPHIA    PA   
12              870 MADISON AVE FRNT 2       NEW YORK    NY   
13                     39 PROGRESS ST          EDISON    NJ   
14          8120 WOODMONT AVE UNIT 960       BETHESDA    MD   
15                       44 W 39TH ST        NEW YORK    NY   
16               135 MADISON AVE FL 10       NEW YORK    NY   
17                       10 WINTER PL          BOSTON    MA   
18                 630 3RD AVE RM 1800   

In [4]:
df['Open/Close'] = ''
df['Google URL'] = ''
df['Google Place ID'] = ''

In [5]:
len(df)

27434

In [6]:
def crawler(i, tempName, tempStreet, tempCity):
    print(i/27434)
    tempJS = findPlace(df.at[i, 'COMPANY'], df.at[i, 'STREET'] + ', ' + df.at[i, 'CITY'], api_key)
    if tempJS['status'] == 'OK' and len(tempJS['candidates']):
        if 'business_status' in tempJS['candidates'][0]:
            df.at[i, 'Open/Close'] = tempJS['candidates'][0]['business_status']
        if 'place_id' in tempJS['candidates'][0]:
            df.at[i, 'Google Place ID'] = tempJS['candidates'][0]['place_id']
        if df.at[i, 'Final_url'] is np.nan and i < 150:#only filling in urls for the first 150 as a sample
            tempDetailsJS = findPlaceDetails(tempJS['candidates'][0]['place_id'], api_key)
            
            if 'website' in tempDetailsJS['result']:
                df.at[i, 'Google URL'] = tempDetailsJS['result']['website']
            else:
                df.at[i, 'Google URL'] = 'Not on Google'

In [7]:
start = time.perf_counter()

with ft.ThreadPoolExecutor(max_workers = 100) as executor:
    executor.map(crawler, df.index, df['COMPANY'], df['STREET'], df['CITY'])
end = time.perf_counter()
print(end-start)

0.0
3.6451119049354813e-057.290223809870963e-050.000109353357148064440.00014580447619741925


0.000182255595246774080.000218706714296128880.0002551578333454837
0.0002916089523948385
0.00032806007144419336

0.000364511190493548160.000400962309542902960.000437413428592257760.0004738645476416126


0.0005103156666909674


0.00076547350003645110.0007290223809870963
0.00069257126193774150.00065612014288838670.00054676678574032230.00061966902383903190.0005832179047896770.00080192461908580590.00083837573813516080.00087482685718451550.00091127797623387040.00094772909528322520.000984180214332580.00102063133338193480.00105708245243128970.0010935335714806445















0.0011299846905299992
0.001166435809579354
0.0012028869286287089
0.00123933804767806370.00127578916672741860.0013486914048261280.0014580447619741926


0.00142159364292483780.00131224028577677340.0013851425238754830.00149449588102354750.00153094700007290210.001567398119122257
0.00160384923817161180.00167675147627032160.0017132025

0.0153094700007290230.017460086024640956



0.016403003572209668
0.0160749435007654720.014215936429248378
0.0125756360720274110.015163665524531604


0.015273018881679668
0.015819785667419990.01363271852445870.0134504629292119260.0142523875482977320.0147262520959393450.0138878763578041840.0141794853101990230.0146168987387912810.0120288692862870890.0139607785959028950.0142888386673470870.014689800976889990.0149814099292848290.0159655901436174080.014835605453087410.0162571990960122470.01636655245316031
0.015127214405482248
0.0150178610483341840.0165852591674564430.0149449588102354740.0163301013341109570.0155281767150251510.016949770357949990.017277830429394180.0134140118101625720.0169862214769993460.0162936502150616040.014543996500692570.016767514762703214
0.0150543121673835380.0146533498578406350.0170591237150980530.0161478457388641830.01476270321498870.0124662827148793470.0143981920244951510.0137420718816067650.0165123569293577320.0150907632864328930.0162207479769628930.0168768681198512




0.0393672085733032
0.038200772763723850.03455566085878836
0.032951811620616750.03648757016840417


0.03513887876357804
0.03783626157323030.0370343369541444940.0349930742873806250.034227600787344170.0364511190493548160.0359772545017132060.033535029525406430.03342567616825837
0.037471750382736750.0366698257636509440.0347379164540351370.0363053145731573950.035284683239775460.040642997740030620.0399868775971422350.03772690821608223
0.03656047240650288
0.039804622001895460.0380185171684770740.037216592549391270.037398848144638040.038455930597069330.0383830283589706240.040424291025734490.035649194430269010.0391120507399577160.0390026973828096550.0334985784063570740.03834657723992126
0.036341765692206750.034920172049281910.0369249835969964260.037617554858934170.036086607858861270.0341182474301961050.041372020121017710.035394036596923530.038674637311365460.036633374644601590.037544652620835460.03823722388277320.039695268644747390.03710723919224320.0393307574542538440.040351388787635780.0367

0.07472479405117737









0.0680906903841948
















0.07064226871764963

0.0734854560034993





0.07308449369395640.073995771670190280.07046001312240285






0.071298388860538020.076219289932200920.075526718670263170.07082452431289640.072756433622512220.072027411241525120.07556316978931253
0.072136764598673170.075162207479769640.06991324633666253

0.07461544069402931



0.0707516220747977
0.074178027265437050.07060581759860028
0.06962163738426770.07086097543194576


0.0703506597652548
0.071954509003426410.069876795217613180.07148064445578480.072428373551068010.072100313479623830.074761245170226730.069366479550922220.07312094481300575
0.074834147408325430.069949697455711890.069111321717576720.071990960122475760.07494350076547350.07687541007508930.077458627979878990.07439673397973318
0.0722096668367719
0.07858861267040898
0.075235109717868340.072574178027265440.074506087336881240.076255741051250270.077567981337027050.07301159145585770.077239921265582850.075891229860756720.0

0.09623095429029671



0.09222133119486768
0.09655901436174090.09120069986148575
0.091747466647226060.0930961580520522


0.09229423343296639
0.094335496099730260.092585842385361230.094299044980680910.093788729313989950.093351315885397680.095647736385507040.09010716629000510.09619450317124736
0.096303856528395420.091929722242472840.090653933075745430.09342421812349640.098235765838011230.097433841218925430.100641539695268650.09644966100459285
0.0920390755996209
0.10078734417146606
0.095574834147408320.094845811766421230.096376758766494120.097543194576073480.099730261719034780.094590653933075750.100422832980972520.099329299409491870.089815557337610260.09929284829044252
0.091601662171028650.092184880075818330.097178683385579930.097105781147481220.093642924837792520.094809360647371870.100714441933367360.091856820004374130.09688707443318510.091018444266238980.097871254647517680.098891885980899610.09761609681417220.092658744623459940.09852737479040606
0.09794415688561639
0.092841000218706710.

...







0.39472916818546330.3941824013997230.4038419479478020.38973536487570170.39553109280454910.396879784209375250.39039148501859006
0.39053728949478750.398702340161842970.3876941022089378

0.3974630021141649





0.398556535685645530.3842312458992491
0.391084056280527840.3945833637092659


0.38933440256615875
0.40001458044761970.389261500328060060.391885980899613630.38725668878034560.38893344025661586
0.381278705256251350.39917620470948456
0.39166727418531750.388204417875628760.380403878399066860.39389079244732810.392177589852008460.3964423707807830.402128745352482330.3913027629948239
0.40096230954290296
0.41018444266238974
0.39348983013778520.394291754756871060.387839906685135240.39651527301888170.39494787489975940.3802945250419188
0.39793686666180650.395968506233141340.39618721294743750.38692862870890140.38623605744696360.39126631187577460.403586790114456530.38918859808996137
0.397827513304658430.40074360282860680.3798935627323759
0.38871473354231970.398447182328497460.4055551505431

0.48312313188014870.470255886855726450.47980608004665740.46642851935554420.47350003645111904
0.4648611212364220.48884595757089744
0.47441131442735290.46890719545090030.463585332069694540.47586935918932710.4835605453087410.478603193118028740.48793467959466360.4755412991178829
0.47973317780855873
0.4970474593570023
0.481482831522927760.47729095283225190.467230443974630040.48060800466574320.4824305606182110.4660275570460013
0.479332215499015830.47459357002259970.47991543340380550.471312969308157750.46489757235547130.47298972078442810.487460815047021930.4692352555223445
0.478639644237078050.482940876284901950.46624626376029743
0.478676095356127430.482138951665816160.4915797914995990.48013414011810160.481191222570532930.48512794342786325
0.47791062185609096
0.47200554057009550.4843260188087774
0.480061237880002930.46927170664139390.463257271998250340.485565356856455470.480352846832397760.48585696580885035


0.475431945760734830.475213239046438750.47404680323685940.48290442516585260.46300211

0.6009695997667128





0.61518553619596120.61157687541007510.6087336881242255






0.61380039367208570.60275570460013120.6148574761245170.59889188598089960.60264635124298320.60450535831450030.602427644528687
0.60990012393380480.6097178683385580.6032660202668222

0.6121600933148648
0.6076037034336954



0.61059269519574260.5880659036232412
0.60257344900488440.5999854195523803


0.6014799154334038
0.60855143252897860.60020412626667640.60691113217175760.5944083983378290.5979441568856164
0.58915943719472190.6174090544579719
0.60217248669534150.59098199314718960.59178391776627550.60534373405263540.60720274112415250.60435955383830290.61245170226725960.6037763359335132
0.6055988918859809
0.6268134431727054
0.60629146314791860.60512502733833930.5972880367427280.60618210979077060.6131442735291973

0.60767660567179420.60078734417146610.60661952321936290.59579354086170440.58901363271852450.59987606619523210.61915870817234090.5918568200043741
0.6068382299336590.61529488955310930.5908361886709922

0.75340817963111470.75078369905956110.76273966610774950.74385798644018370.75264270613107820.74764890282131660.7556316978931253
0.75300721732157170.75308011955967050.7480863162499088

0.7582561784646789
0.7502369322738208



0.75986002770285040.7300794634395276
0.75461106655974340.7450973244878618


0.7490704964642414
0.75636072027411240.74334767077349270.7505649923452650.74057738572574180.7463366625355399
0.73992126558285340.7615367791791208
0.75421010425020050.73653131151126340.73685937158270750.75041918786906760.74936210541663630.75246045053583140.75614201355981630.7484143763213531
0.753335277393016
0.7691915141794853
0.75402784865495380.75071079682146250.74236349055916020.74848727855945170.7594955165123569

0.74659182036888530.74422249763067730.74801341401181020.73755194284464530.73656776263031280.74342057301159150.76642122913173440.7386090252970766
0.75020048115477140.76040679448859080.7376248450827441

0.7635051396077860.76193774148866370.75060144346431430.76088065903623240.760771

0.97579645695122840.96792301523656780.98115477145148360.9556025369978859
0.96985492454618360.9676314062841729
0.96989137566523290.97371874316541520.9720055405700955

0.9729897207844281
0.9650069257126194



0.97831158416563390.9412043449733907
0.972880367427280.9652256324269155


0.9678136618794198
0.97291681854632940.96004957352190710.97517678792738940.9587008821170810.9597944156885616
0.9655901436174090.9783844864037325
0.97368229204636580.95913829554567320.95421739447401030.96679303054603770.97113071371291090.97834803528468330.97885835095137420.9680323685937158
0.9707662025224174
0.9871692060946271
0.97630677261791930.97065684916526940.96070569366479550.96529853466501430.9742290588321062

0.96475176787927390.96008602464095650.95778960414084710.95655026609316910.95388933440256620.96103375373623970.99030400233287160.9522854851643946
0.96511627906976750.98487278559451770.9644601589268791

0.97918641102281840.97882189983232480.96759495516512360.97539549464168550.9843989210468761
0.97324

In [11]:
print(end-start)

306.29239537690756


In [10]:
df.to_csv('VZ_leads_MM_MB_COMBINED_3P_Google.csv')